In [1]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\neteera\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\neteera\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import sklearn
import math
from datetime import datetime
import pandas as pd

In [4]:
import math
from datetime import datetime
import pandas as pd

# Haversine function to calculate the distance between two points on the Earth's surface
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Earth radius in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Output is the shortest distance between 2 points (in kilometers)

# User class to store carpooling participant information
class User:
    def __init__(self, user_id, name, start_lon, start_lat, end_lon, end_lat, start_point_name, end_point_name, distance, travel_time, start_time, end_time):
        self.user_id = user_id
        self.name = name
        self.start_lon = start_lon
        self.start_lat = start_lat
        self.end_lon = end_lon
        self.end_lat = end_lat
        self.start_point_name = start_point_name
        self.end_point_name = end_point_name
        self.distance = distance  # distance willing to travel from start to end
        self.travel_time = travel_time
        self.start_time = datetime.strptime(start_time, '%H:%M')  # Start time as HH:MM
        self.end_time = datetime.strptime(end_time, '%H:%M')  # End time as HH:MM

    def __repr__(self):
        return f"User({self.name})"

# Matching function to find potential carpool matches  
# loop2 times for finding 1 person to another user that can matching 
def find_matches(users, distance_threshold=1.0, time_threshold=30):
    matches = []
    
    for i, user1 in enumerate(users):
        for user2 in users[i+1:]:
            start_distance = haversine(user1.start_lon, user1.start_lat, user2.start_lon, user2.start_lat)
            end_distance = haversine(user1.end_lon, user1.end_lat, user2.end_lon, user2.end_lat)
            
            if start_distance <= distance_threshold and end_distance <= distance_threshold:
                # Check if travel time overlap is within threshold
                start_time_diff = abs((user1.start_time - user2.start_time).total_seconds() / 60)
                end_time_diff = abs((user1.end_time - user2.end_time).total_seconds() / 60)
                
                if start_time_diff <= time_threshold and end_time_diff <= time_threshold:
                    match_time = (max(user1.start_time, user2.start_time), min(user1.end_time, user2.end_time))
                    matches.append({
                        'user_id_person1': user1.user_id,
                        'person1_name': user1.name,
                        'user_id_person2': user2.user_id,
                        'person2_name': user2.name,
                        'start_latitude': user1.start_lat,
                        'start_longitude': user1.start_lon,
                        'start_point_name': user1.start_point_name,
                        'end_latitude': user1.end_lat,
                        'end_longitude': user1.end_lon,
                        'end_point_name': user1.end_point_name,
                        'start_time': user1.start_time.strftime('%H:%M'),
                        'end_time': user1.end_time.strftime('%H:%M'),
                        'match_time': f"{match_time[0].strftime('%H:%M')} - {match_time[1].strftime('%H:%M')}"
                    })
    
    return pd.DataFrame(matches)

# Example usage
users = [
    User(1, "Alice", 100.0, 13.7, 100.2, 13.8, "Start A", "End A", 5, 30, "08:00", "09:00"),
    User(2, "Bob", 100.1, 13.75, 100.25, 13.85, "Start B", "End B", 6, 35, "08:15", "09:15"),
    User(3, "Charlie", 101.0, 14.0, 101.1, 14.1, "Start C", "End C", 8, 45, "07:45", "08:45"),
]

matches_df = find_matches(users, distance_threshold=13.0, time_threshold=60)

# Display the resulting DataFrame
print(matches_df)


   user_id_person1 person1_name  user_id_person2 person2_name  start_latitude  \
0                1        Alice                2          Bob            13.7   

   start_longitude start_point_name  end_latitude  end_longitude  \
0            100.0          Start A          13.8          100.2   

  end_point_name start_time end_time     match_time  
0          End A      08:00    09:00  08:15 - 09:00  
